# DF with Netflix Titles

## Setup

In [ ]:
!pip install pyspark

from pyspark.sql import SparkSession
from pyspark.sql import functions as F

spark = (
    SparkSession
    .builder
    .master('local[*]')
    .appName('myapp')
    .getOrCreate()
)

!wget https://tinyurl.com/pyspark-netflix-csv -O netflix.csv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285388 sha256=d848e6000683bf901db58eecfc3fe0a3c793ef5abc145d68e8330544faa8f5fd
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark
--2023-08-22 09:22:13--  https://tinyurl.com/pyspark-netflix-csv
Resolving tinyurl.com (tinyurl.com)... 172.67.1.225, 104.20.138.65, 104.20.139.65, ...
Connecting to tinyurl.com (tinyurl.com)|172.67.1.225|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://raw.githubusercontent.com/justincheu-ng/misc/master/netflix.csv [following]
--2023-08-22 09:22:13--  https://raw.githubusercontent.com/justincheu-ng/misc/master/netflix.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.19

In [ ]:
df = (
    spark.read
         .csv('netflix.csv',header=True,inferSchema=True)
         .withColumn('date_added', F.trim('date_added'))
         .withColumn('date_added', F.to_date('date_added', format="MMMM d, yyyy"))
         .dropna()
)
df.show()

+-------+-------+--------------------+-------------------+--------------------+--------------------+----------+------------+------+---------+--------------------+--------------------+
|show_id|   type|               title|           director|                cast|             country|date_added|release_year|rating| duration|           listed_in|         description|
+-------+-------+--------------------+-------------------+--------------------+--------------------+----------+------------+------+---------+--------------------+--------------------+
|     s8|  Movie|             Sankofa|       Haile Gerima|Kofi Ghanaba, Oya...|United States, Gh...|2021-09-24|        1993| TV-MA|  125 min|Dramas, Independe...|On a photo shoot ...|
|     s9|TV Show|The Great British...|    Andy Devonshire|Mel Giedroyc, Sue...|      United Kingdom|2021-09-24|        2021| TV-14|9 Seasons|British TV Shows,...|A talented batch ...|
|    s10|  Movie|        The Starling|     Theodore Melfi|Melissa McCarthy,...| 

## Task

### 1. Count the number of movies and TV Show

In [ ]:
# your code

df.count()

5316

### 2. List all the directors

In [ ]:
# your code
(
  df.select('director')
    .distinct()
    .show()
)

+--------------------+
|            director|
+--------------------+
|           Eric Meza|
|          John Wells|
|         Erol Özlevi|
|   Jennifer Westcott|
|       Neeraj Pandey|
|   Wash Westmoreland|
|       Patricia Font|
|          Bobby Roth|
|      Aleksey German|
| Rhiannon Bannenberg|
|      Elizabeth Wood|
|          Manish Jha|
|        April Mullen|
|Harry Elfont, Deb...|
|          Will Gluck|
|Jesús Magaña Vázquez|
|      Gail Willumsen|
|Jacques Perrin, J...|
|         John Milius|
|        Pramod Pawar|
+--------------------+
only showing top 20 rows



### 3. Which title has the longest description?

In [ ]:
# your code
max_length =(
 df.select(F.length(F.col('description')))
   .agg({"length(description)":"max"})
   .collect()[0][0]
)

(
  df.filter(F.length(F.col("description")) == max_length)
    .show()
)

+-------+-----+-----+------------+--------------------+-------+----------+------------+------+--------+--------------------+--------------------+
|show_id| type|title|    director|                cast|country|date_added|release_year|rating|duration|           listed_in|         description|
+-------+-----+-----+------------+--------------------+-------+----------+------------+------+--------+--------------------+--------------------+
|  s4949|Movie|Billu|Priyadarshan|Irrfan Khan, Shah...|  India|2018-04-01|        2009| TV-PG| 132 min|Dramas, Internati...|An Indian village...|
+-------+-----+-----+------------+--------------------+-------+----------+------------+------+--------+--------------------+--------------------+



In [ ]:
df.groupBy('show_id').agg(F.max(F.length('description')).alias('max_length')) \
.orderBy(F.col('max_length').desc()).limit(1).show()

+-------+----------+
|show_id|max_length|
+-------+----------+
|  s4949|       248|
+-------+----------+



### 4. Create a pivot table on "year" against "type"

In [ ]:
# Your code

df.groupBy('release_year').pivot('type').count().show()

+------------+-----+-------+
|release_year|Movie|TV Show|
+------------+-----+-------+
|        1987|    8|   null|
|        1956|    2|   null|
|        2016|  566|     10|
|        2020|  408|     33|
|        2012|  159|      4|
|        1958|    3|   null|
|        1972|    4|   null|
|        1988|   15|   null|
|        2019|  492|     24|
|        2017|  638|     17|
|        1977|    5|   null|
|        2014|  234|      8|
|        1971|    5|   null|
|        1984|   11|   null|
|        1982|   15|   null|
|        2013|  195|      2|
|        2005|   65|   null|
|        2000|   32|   null|
|        1965|    2|   null|
|        1962|    3|   null|
+------------+-----+-------+
only showing top 20 rows



### 5. Create a new column to calculate years between release year and year added to Netflix.

In [ ]:
# Your code

df = df.withColumn('gap', F.year(df.date_added) - df.release_year)
df.select(['release_year', 'date_added', 'gap']).show()

+------------+----------+----+
|release_year|date_added| gap|
+------------+----------+----+
|        1993|2021-09-24|28.0|
|        2021|2021-09-24| 0.0|
|        2021|2021-09-24| 0.0|
|        2021|2021-09-23| 0.0|
|        1998|2021-09-21|23.0|
|        2010|2021-09-20|11.0|
|        2013|2021-09-19| 8.0|
|        2013|2021-09-19| 8.0|
|        2017|2021-09-16| 4.0|
|        1975|2021-09-16|46.0|
|        1978|2021-09-16|43.0|
|        1983|2021-09-16|38.0|
|        1987|2021-09-16|34.0|
|        2012|2021-09-16| 9.0|
|        2001|2021-09-16|20.0|
|        2002|2021-09-15|19.0|
|        2003|2021-09-15|18.0|
|        2004|2021-09-15|17.0|
|        2001|2021-09-15|20.0|
|        2011|2021-09-15|10.0|
+------------+----------+----+
only showing top 20 rows



### 6. How many movie has Steven Spielberg directed?

In [ ]:
# Your code

df.filter(df.director == 'Steven Spielberg').count()

11

### 7. Which actress/actor has been casted in Comedies the most?

In [ ]:
# Your code

df1 = df.filter(F.col('listed_in').like('%Comedies%'))

df1 = df1.withColumn('cast_exploded', F.explode(F.split('cast', ', ')))

df1 = df1.groupBy('cast_exploded').count().orderBy(F.col("count").desc()).limit(1)

df1.show()

+-------------+-----+
|cast_exploded|count|
+-------------+-----+
|  Anupam Kher|   20|
+-------------+-----+



### 8. Create a pivot table for country against categories

In [ ]:
# Your code

df2 = df.withColumn('country_exploded', F.explode(F.split('country', ', ')))
df2 = df2.withColumn('listed_in_exploded', F.explode(F.split('listed_in', ', ')))
df2.groupBy('country_exploded').pivot('listed_in_exploded').count().show()

+----------------+------------------+--------------+------------+----------------+------------------------+-----------------+--------------+--------+--------------+-----------+-------------+----------+------+--------------------+-------------+------------------+--------------------+----------------------+--------+---------------+------------+------+----------------+----------+---------------+-----------------+----------------+-------------------+-------------------------+-------------+---------------+----------------------------+---------------------+-----------+---------+---------+------------+-------------------+--------+------------+-------------+---------+
|country_exploded|Action & Adventure|Anime Features|Anime Series|British TV Shows|Children & Family Movies|Classic & Cult TV|Classic Movies|Comedies|Crime TV Shows|Cult Movies|Documentaries|Docuseries|Dramas|Faith & Spirituality|Horror Movies|Independent Movies|International Movies|International TV Shows|Kids' TV|Korean TV Shows|